### imports

In [ ]:
import csv
import pandas as pd
from math import floor
from functools import reduce

### read in csv source into a pandas dataframe

In [ ]:
csv_path = 'files/full.csv'
df = pd.read_csv(csv_path)
df.head(40)

### get count by iso3 code column

In [ ]:
dfc = dict(df.groupby(['ISO3']).count())
iso3_count = dict(dfc['OBJECTID'])
iso3_count

### create a stats dataframe with ISO3 code to join back to the end result
- filter out records that have zeros in the STATUS_YR col
- use the `describe()` method to get statistics
- filter out the columns to just mean, min, & max
- convert the column types to integers
- rename the columns for the final join to the output

In [ ]:
temp_stats = df.loc[df['STATUS_YR'] > 0].groupby(['ISO3']).describe()['STATUS_YR'][['mean','min','max']]
temp_stats = temp_stats.astype({'mean': 'int32','min': 'int32','max': 'int32'}, inplace=True)
df_stats = temp_stats.rename(columns={'mean':'StatsYearAverage', 'max': 'StatsYearMax', 'min': 'StatsYearMin'})
df_stats

### setup the fields config

In [ ]:
fields = [
    {'name':'IUCN_CAT', 'prefix': 'IUCNCategory'},
    {'name':'OWN_TYPE', 'prefix': 'OwnerType'},
    {'name':'DESIG_TYPE', 'prefix': 'DesignType'},
    {'name':'STATUS', 'prefix': 'Status'},
    {'name':'GOV_TYPE', 'prefix': 'GovernmentType'}
]

### process the wdpa dataframe
- create a crosstab dataframe for each `field` in the `fields` config giving us summary stats with unique values of fields for columns
- join the protected area count
- join the statistics

merge multiple dataframes
https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

In [ ]:
df_holder = []
df1 = None
output_merged = None
for field in fields:
    df1 = pd.crosstab(df.ISO3, df[field['name']]).reset_index()
    for ind,col in enumerate(df1.columns):
    # print (ind, col)
        if ind > 0:
            new_name = '{}_{}'.format(field['prefix'], col).replace(' ','_').replace('-', '_')
            df1.rename(columns={col:new_name}, inplace=True)
    
    df_holder.append(df1)

output_merged = reduce(lambda left, right: pd.merge(left, right, on=['ISO3'], how='outer'), df_holder)
output_merged['Protected_Areas'] = output_merged['ISO3'].map(iso3_count)

final_output = reduce(lambda left, right: pd.merge(left, right, on=['ISO3'], how='outer'), [output_merged, df_stats])
final_output.head(25)

### export final result to CSV

In [ ]:
final_output.to_csv('files/processed.csv')